word, phrase, set of characters, characters->string

i want to get the first 10 example prompt from the dataset which have 'word' in it, and replace 'word' with 'string'
and do the same for phrases, set of characters, and characters.


In [4]:
import re
import spacy
import pandas as pd
from datasets import load_dataset

ds = load_dataset("wellesley-easel/StudentEval",split='test')
# Access the text prompts
# prompts = ds['submitted_text']

# Initialize spaCy model for lemmatization and NLP tasks
nlp = spacy.load("en_core_web_trf")

# Define verb categories
Verb_categories = ['return', 'concatenate', 'loop', 'skip', 'sort', 'reverse', 'typecast', 'replace', 'remove']



In [19]:
#make prompts only the ones where it is either the first attempt or the last attempt
df = pd.DataFrame(ds)

# Filter rows where either 'first_attempt' or 'last_attempt' is True
filtered_df = df[(df['first_attempt'] == True) | (df['last_attempt'] == True)]

# Access the 'submitted_text' from the filtered DataFrame
prompts = filtered_df['prompt'].tolist()


In [18]:
import re
import pandas as pd

# Example DataFrame setup
df = pd.DataFrame(ds)

# Step 1: Filter rows where either 'first_attempt' or 'last_attempt' is True
filtered_df = df[(df['first_attempt'] == True) | (df['last_attempt'] == True)]

# Step 2: Create two splits
original_split = filtered_df.copy()
modified_split = filtered_df.copy()

# Step 3: Define the search and replace terms
search_term = 'input'
replace_term = 'parameter'

# Define a function to replace "word" with "string" in prompts
def replace_word_in_prompt_noun(prompt, search_term, replace_term):
    # Compile the pattern for the search term with optional suffix handling
    term_pattern = re.compile(rf"\b({re.escape(search_term)})(s)?\b", flags=re.IGNORECASE)
    
    modified_prompt = prompt  # Start with the original prompt
    matches = list(term_pattern.finditer(modified_prompt))
    
    # Only modify if there are matches
    if matches:
        print("Original:", prompt)
        for match in reversed(matches):
            matched_term = match.group(1)  # The matched string
            suffix = match.group(2)
            print("Suffix:", suffix)
            start_pos = match.start()
            end_pos = match.end()
            
            replace_with = replace_term
            if matched_term.istitle():
                # Capitalize the replacement term if the matched term is capitalized
                replace_with = replace_term.capitalize()
            if suffix:
                # Append the suffix to the replacement term if it exists
                replace_with += suffix
            
            # Replace the matched term with the replacement string
            modified_prompt = modified_prompt[:start_pos] + replace_with + modified_prompt[end_pos:]
        print("Modified:", modified_prompt)
    return modified_prompt

# Step 4: Apply the replacement to each prompt in the modified split
modified_split['modified_prompt'] = modified_split['prompt'].apply(replace_word_in_prompt_noun, search_term=search_term, replace_term=replace_term)

# Step 5: Keep only rows where the prompt was actually changed
# Compare the original and modified prompts to filter rows where changes occurred
modified_split = modified_split[modified_split['prompt'] != modified_split['modified_prompt']]

modified_split['prompt'] = modified_split['modified_prompt']
modified_split = modified_split.drop(columns=['modified_prompt'])

matched_ids = modified_split['__index_level_0__'].tolist()

#filter the original split to only include the matched ids
original_split = original_split[original_split['__index_level_0__'].isin(matched_ids)]

original_split = original_split.drop(columns=['__index_level_0__'])
modified_split = modified_split.drop(columns=['__index_level_0__'])

Original: def add_up(arr):
    """
    Input: list of items, that may be a combination of strings and integers. Function takes the sum of the integers and outputs it
    """
    
Suffix: None
Modified: def add_up(arr):
    """
    Parameter: list of items, that may be a combination of strings and integers. Function takes the sum of the integers and outputs it
    """
    
Original: def add_up(arr):
    """
    Input: list of strings. Function checks each item in the list to check if it is a number character. If it is a number, convert it to an integer and it adds it to a new list. If it is not a number integer it ignores it. Outputs the sum of each ingter in the new list. If the original list contains no integers, return 0
    """
    
Suffix: None
Modified: def add_up(arr):
    """
    Parameter: list of strings. Function checks each item in the list to check if it is a number character. If it is a number, convert it to an integer and it adds it to a new list. If it is not a number in

In [19]:
# upload a dataset to huggingface
import os
# a dataset with two splits: train(original), test(modified)
from datasets import Dataset, DatasetDict
# Convert DataFrames to Hugging Face datasets
original_dataset = Dataset.from_pandas(original_split)
modified_dataset = Dataset.from_pandas(modified_split)

# Create a dataset dictionary with splits
dataset_dict = DatasetDict({
    'train': original_dataset,
    'test': modified_dataset
})

# Save the dataset dictionary to disk
# dataset_dict.save_to_disk('temporary_dataset_wordtostring')
#upload to nuprl-staging
dataset_dict.push_to_hub("nuprl-staging/temporary_dataset_inputtoparameter", private=True, token=os.getenv('HF_TOKEN_WRITE'))


Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  3.11it/s]


CommitInfo(commit_url='https://huggingface.co/datasets/nuprl-staging/temporary_dataset_inputtoparameter/commit/6ecc0d34fffd043abd23a95a723e2c8a33df40da', commit_message='Upload dataset', commit_description='', oid='6ecc0d34fffd043abd23a95a723e2c8a33df40da', pr_url=None, pr_revision=None, pr_num=None)

In [65]:
import re
import pandas as pd

# Example DataFrame setup
# df = pd.DataFrame(ds)

# Step 1: Filter rows where either 'first_attempt' or 'last_attempt' is True
filtered_df = df[(df['first_attempt'] == True) | (df['last_attempt'] == True)]

# Step 2: Create two splits
original_split = filtered_df.copy()
modified_split = filtered_df.copy()

# Step 3: Define the search and replace terms
search_term = 'append'
replace_term = 'concatenate'

# Define a function to replace "word" with "string" in prompts
def replace_word_in_prompt_verb(prompt, search_term, replace_term):
    # Compile the pattern for the search term with optional suffix handling
    if search_term.endswith("e"):
        term_pattern = re.compile(rf"\b({re.escape(search_term)}|{re.escape(search_term[:-1])})(es|ed|ing)\b", flags=re.IGNORECASE)
    else:
        term_pattern = re.compile(rf"\b({re.escape(search_term)})(s|ed|ing)?\b", flags=re.IGNORECASE)
    modified_prompt = prompt  # Start with the original prompt
    matches = list(term_pattern.finditer(modified_prompt))
    
    # Only modify if there are matches
    if matches:
        print('Original:', modified_prompt)
        for match in reversed(matches):
            matched_term = match.group(1)  # The matched string
            suffix = match.group(2)
            print('Suffix:', suffix)
            start_pos = match.start()
            end_pos = match.end()
            
            replace_with = replace_term
            if matched_term.istitle():
                # Capitalize the replacement term if the matched term is capitalized
                replace_with = replace_term.capitalize()
            if suffix:
                # Append the suffix to the replacement term if it exists
                replace_with += suffix
            
            # Replace the matched term with the replacement string
            modified_prompt = modified_prompt[:start_pos] + replace_with + modified_prompt[end_pos:]
        print('Modified:', modified_prompt)
    return modified_prompt

modified_split['modified_prompt'] = modified_split['prompt'].apply(replace_word_in_prompt_noun, search_term=search_term, replace_term=replace_term)

# Step 5: Keep only rows where the prompt was actually changed
modified_split = modified_split[modified_split['prompt'] != modified_split['modified_prompt']]
modified_split['prompt'] = modified_split['modified_prompt']
modified_split = modified_split.drop(columns=['modified_prompt'])

matched_ids = modified_split['__index_level_0__'].tolist()

#filter the original split to only include the matched ids
original_split = original_split[original_split['__index_level_0__'].isin(matched_ids)]

original_split = original_split.drop(columns=['__index_level_0__'])
modified_split = modified_split.drop(columns=['__index_level_0__'])

#For print out to see purpose
# prompts = original_split['prompt'].tolist()
# for prompt in prompts:
#     replace_word_in_prompt_verb(prompt, search_term, replace_term)

In [63]:
# For print out to see purpose
prompts = original_split['prompt'].tolist()
for prompt in prompts:
    replace_word_in_prompt_verb(prompt, search_term, replace_term)

Original: def add_int(lst, num):
    """
    For each element in the list lst, if the element is a string, append the number num to the end of the string. If the element is an integer, add num to it. If it is any other type, do not do anything to it.

    """
    
Suffix: None
Modified: def add_int(lst, num):
    """
    For each element in the list lst, if the element is a string, concatenate the number num to the end of the string. If the element is an integer, add num to it. If it is any other type, do not do anything to it.

    """
    
Original: def add_int(lst, num):
    """
    Append the input num to the end of any strings, and add the input num to any numbers in a list. If there is a list within a list, ignore it.
    """
    
Suffix: None
Modified: def add_int(lst, num):
    """
    Concatenate the input num to the end of any strings, and add the input num to any numbers in a list. If there is a list within a list, ignore it.
    """
    
Original: def add_int(lst, num):
   

In [67]:
# upload a dataset to huggingface
import os
# a dataset with two splits: train(original), test(modified)
from datasets import Dataset, DatasetDict
# Convert DataFrames to Hugging Face datasets
original_dataset = Dataset.from_pandas(original_split)
modified_dataset = Dataset.from_pandas(modified_split)

# Create a dataset dictionary with splits
dataset_dict = DatasetDict({
    'train': original_dataset,
    'test': modified_dataset
})

# Save the dataset dictionary to disk
# dataset_dict.save_to_disk('temporary_dataset_wordtostring')
#upload to nuprl-staging
dataset_dict.push_to_hub("nuprl-staging/temporary_dataset_appendtoconcatenate", private=True, token=os.getenv('HF_TOKEN_WRITE'))


Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  2.64it/s]


CommitInfo(commit_url='https://huggingface.co/datasets/nuprl-staging/temporary_dataset_appendtoconcatenate/commit/b6e47c6ab1030e744599d1dfa6f6807f453dfc59', commit_message='Upload dataset', commit_description='', oid='b6e47c6ab1030e744599d1dfa6f6807f453dfc59', pr_url=None, pr_revision=None, pr_num=None)

In [26]:
#load the dataset
from datasets import load_dataset
dataset = load_dataset('nuprl-staging/temporary_dataset_appendtoconcatenate')
print(dataset['train'][1]['prompt'])
print(dataset['test'][1]['prompt'])

def add_int(lst, num):
    """
    Append the input num to the end of any strings, and add the input num to any numbers in a list. If there is a list within a list, ignore it.
    """
    
def add_int(lst, num):
    """
    Concatenate the input num to the end of any strings, and add the input num to any numbers in a list. If there is a list within a list, ignore it.
    """
    


In [63]:
# Print the dataset
for i, example in enumerate(dataset['train']):
    print(f"Example {i}:", example['prompt'])

Example 0: def meeps_morps(items):
    """
    take in a list of strings and output a count of how many times the word 'Meep' and 'Morp' occur
    """
    
Example 1: def meeps_morps(items):
    """
    take in a list of strings and output a list count of how many times the word 'Meep' and 'Morp' occur
    """
    
Example 2: def mod_end(words, ending):
    """
    Modify each word in the list by taking the ending substring and replacing it with the letter s
    """
    
Example 3: def mod_end(words, ending):
    """
    Takes a list of strings words and a string ending, removes any instance of ending from each item in words, and replaces it with an 's'.
    """
    
Example 4: def mod_end(words, ending):
    """
    Take a list of words sharing the same ending, and a string as an input.
Search the last letters of the word for the string, and replace it with 's'.
    """
    
Example 5: def mod_end(words, ending):
    """
    For each word in the list, remove the last instance of endin